In [3]:
language_key = "8dcXJ41LuAG6wTmqYLWoOAvyDH4JSwHowSBU6VICBhgWUU0LAmrdJQQJ99BFACi5YpzXJ3w3AAAaACOG3smr"
language_endpoint = "https://1sesac035-language.cognitiveservices.azure.com/"

### named entity recognition

In [4]:
# This example requires environment variables named "LANGUAGE_KEY" and "LANGUAGE_ENDPOINT"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=language_endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example function for recognizing entities from text
def entity_recognition_example(client):

    try:
        documents = ["I had a wonderful trip to Seattle last week."]
        result = client.recognize_entities(documents = documents)[0]

        print("Named Entities:\n")
        for entity in result.entities:
            print("\tText: \t", entity.text, "\tCategory: \t", entity.category, "\tSubCategory: \t", entity.subcategory,
                    "\n\tConfidence Score: \t", round(entity.confidence_score, 2), "\tLength: \t", entity.length, "\tOffset: \t", entity.offset, "\n")

    except Exception as err:
        print("Encountered exception. {}".format(err))
entity_recognition_example(client)

Named Entities:

	Text: 	 trip 	Category: 	 Event 	SubCategory: 	 None 
	Confidence Score: 	 0.66 	Length: 	 4 	Offset: 	 18 

	Text: 	 Seattle 	Category: 	 Location 	SubCategory: 	 City 
	Confidence Score: 	 1.0 	Length: 	 7 	Offset: 	 26 

	Text: 	 last week 	Category: 	 DateTime 	SubCategory: 	 DateRange 
	Confidence Score: 	 1.0 	Length: 	 9 	Offset: 	 34 



### emotion recognition

In [5]:
def sentiment_analysis_with_opinion_mining_example(client):

    documents = [
        "The food and service were unacceptable. The concierge was nice, however."
    ]

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    positive_reviews = [doc for doc in doc_result if doc.sentiment == "positive"]
    negative_reviews = [doc for doc in doc_result if doc.sentiment == "negative"]

    positive_mined_opinions = []
    mixed_mined_opinions = []
    negative_mined_opinions = []

    for document in doc_result:
        print("Document Sentiment: {}".format(document.sentiment))
        print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            document.confidence_scores.positive,
            document.confidence_scores.neutral,
            document.confidence_scores.negative,
        ))
        for sentence in document.sentences:
            print("Sentence: {}".format(sentence.text))
            print("Sentence sentiment: {}".format(sentence.sentiment))
            print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                print("......'{}' target '{}'".format(target.sentiment, target.text))
                print("......Target score:\n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
                    target.confidence_scores.positive,
                    target.confidence_scores.negative,
                ))
                for assessment in mined_opinion.assessments:
                    print("......'{}' assessment '{}'".format(assessment.sentiment, assessment.text))
                    print("......Assessment score:\n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative,
                    ))
            print("\n")
        print("\n")
          
sentiment_analysis_with_opinion_mining_example(client)

Document Sentiment: negative
Overall scores: positive=0.00; neutral=0.00; negative=1.00 

Sentence: The food and service were unacceptable. 
Sentence sentiment: negative
Sentence score:
Positive=0.00
Neutral=0.00
Negative=1.00

......'negative' target 'food'
......Target score:
......Positive=0.01
......Negative=0.99

......'negative' assessment 'unacceptable'
......Assessment score:
......Positive=0.01
......Negative=0.99

......'negative' target 'service'
......Target score:
......Positive=0.01
......Negative=0.99

......'negative' assessment 'unacceptable'
......Assessment score:
......Positive=0.01
......Negative=0.99



Sentence: The concierge was nice, however.
Sentence sentiment: neutral
Sentence score:
Positive=0.22
Neutral=0.75
Negative=0.04

......'positive' target 'concierge'
......Target score:
......Positive=1.00
......Negative=0.00

......'positive' assessment 'nice'
......Assessment score:
......Positive=1.00
......Negative=0.00







### language detection

In [6]:
def language_detection_example(client):
    try:
        documents = ["Ce document est rédigé en Français."]
        response = client.detect_language(documents = documents, country_hint = 'us')[0]
        print("Language: ", response.primary_language.name)

    except Exception as err:
        print("Encountered exception. {}".format(err))
language_detection_example(client)

Language:  French


### extractive summarization

In [7]:
def sample_extractive_summarization(client):
    import os
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import (
        TextAnalyticsClient,
        ExtractiveSummaryAction
    ) 

    document = [
        "The extractive summarization feature uses natural language processing techniques to locate key sentences in an unstructured text document. "
        "These sentences collectively convey the main idea of the document. This feature is provided as an API for developers. " 
        "They can use it to build intelligent solutions based on the relevant information extracted to support various use cases. "
        "Extractive summarization supports several languages. It is based on pretrained multilingual transformer models, part of our quest for holistic representations. "
        "It draws its strength from transfer learning across monolingual and harness the shared nature of languages to produce models of improved quality and efficiency. "
    ]

    poller = client.begin_analyze_actions(
        document,
        actions=[
            ExtractiveSummaryAction(max_sentence_count=4)
        ],
    )

    document_results = poller.result()
    for result in document_results:
        extract_summary_result = result[0]  # first document, first result
        if extract_summary_result.is_error:
            print("...Is an error with code '{}' and message '{}'".format(
                extract_summary_result.code, extract_summary_result.message
            ))
        else:
            print("Summary extracted: \n{}".format(
                " ".join([sentence.text for sentence in extract_summary_result.sentences]))
            )

sample_extractive_summarization(client)

Summary extracted: 
The extractive summarization feature uses natural language processing techniques to locate key sentences in an unstructured text document. This feature is provided as an API for developers. Extractive summarization supports several languages. It is based on pretrained multilingual transformer models, part of our quest for holistic representations.


### abstract summarization

In [13]:
def sample_abstractive_summarization() -> None:
    
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient

    document = [
        "At Microsoft, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic, "
        "human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Cognitive "
        "Services, I have been working with a team of amazing scientists and engineers to turn this quest into a "
        "reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of "
        "human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the "
        "intersection of all three, there's magic-what we call XYZ-code as illustrated in Figure 1-a joint "
        "representation to create more powerful AI that can speak, hear, see, and understand humans better. "
        "We believe XYZ-code will enable us to fulfill our long-term vision: cross-domain transfer learning, "
        "spanning modalities and languages. The goal is to have pretrained models that can jointly learn "
        "representations to support a broad range of downstream AI tasks, much in the way humans do today. "
        "Over the past five years, we have achieved human performance on benchmarks in conversational speech "
        "recognition, machine translation, conversational question answering, machine reading comprehension, "
        "and image captioning. These five breakthroughs provided us with strong signals toward our more ambitious "
        "aspiration to produce a leap in AI capabilities, achieving multisensory and multilingual learning that "
        "is closer in line with how humans learn and understand. I believe the joint XYZ-code is a foundational "
        "component of this aspiration, if grounded with external knowledge sources in the downstream AI tasks."
    ]

    poller = client.begin_abstract_summary(document)
    abstract_summary_results = poller.result()
    for result in abstract_summary_results:
        if result.kind == "AbstractiveSummarization":
            print("Summaries abstracted:")
            [print(f"{summary.text}\n") for summary in result.summaries]
        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))

if __name__ == "__main__":
    sample_abstractive_summarization()

Summaries abstracted:
The Chief Technology Officer of Azure AI Cognitive Services discusses Microsoft's commitment to advancing AI by integrating monolingual text, audio or visual signals, and multilingual capabilities into a unified approach, termed XYZ-code. This innovative framework aims to create AI that can better understand and communicate across various modalities and languages, akin to human cognition. Through their efforts, Microsoft has achieved human-level performance in several key AI domains, including speech recognition, machine translation, and image captioning. The ultimate goal is to develop pretrained models that can learn from multiple sources and transfer knowledge across tasks, emulating the breadth of human learning. The XYZ-code is seen as a critical element in realizing this vision, with the potential to revolutionize AI's ability to process and understand diverse types of information. This approach not only demonstrates current progress but also sets a directio

### custom question answering

In [19]:
# curl -X POST "https://1sesac035-language.cognitiveservices.azure.com/language/:query-knowledgebases?projectName=QnAProj&api-version=2021-10-01&deploymentName=production" -H "Ocp-Apim-Subscription-Key: 8dcXJ41LuAG6wTmqYLWoOAvyDH4JSwHowSBU6VICBhgWUU0LAmrdJQQJ99BFACi5YpzXJ3w3AAAaACOG3smr" -H "Content-Type: application/json" -d "{\"top\":3,\"question\":\"YOUR_QUESTION_HERE\",\"includeUnstructuredSources\":true,\"confidenceScoreThreshold\":\"YOUR_SCORE_THRESHOLD_HERE\",\"answerSpanRequest\":{\"enable\":true,\"topAnswersWithSpan\":1,\"confidenceScoreThreshold\":\"YOUR_SCORE_THRESHOLD_HERE\"},\"filters\":{\"metadataFilter\":{\"logicalOperation\":\"YOUR_LOGICAL_OPERATION_HERE\",\"metadata\":[{\"key\":\"YOUR_ADDITIONAL_PROP_KEY_HERE\",\"value\":\"YOUR_ADDITIONAL_PROP_VALUE_HERE\"}]}}}"

In [17]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient

endpoint = "https://1sesac035-language.cognitiveservices.azure.com/"
credential = AzureKeyCredential("8dcXJ41LuAG6wTmqYLWoOAvyDH4JSwHowSBU6VICBhgWUU0LAmrdJQQJ99BFACi5YpzXJ3w3AAAaACOG3smr")
knowledge_base_project = "QnAProj"
deployment = "production"

def main():
    client = QuestionAnsweringClient(endpoint, credential)
    with client:
        question="비용은 얼마인가요?"
        output = client.get_answers(
            question = question,
            project_name=knowledge_base_project,
            deployment_name=deployment
        )
    print("Q: {}".format(question))
    print("A: {}".format(output.answers[0].answer))

if __name__ == '__main__':
    main()

Q: 비용은 얼마인가요?
A: Microsoft Learn 교육은 완전히 무료이며 Microsoft 제품에 관심이 있는 사람이라면 누구나 사용할 수 있습니다.


### microsoft azure translator

In [22]:
import requests, uuid, json

# Add your key and endpoint
key = "5BOOH90Qyj7T9opgcLQs1dFZWcfHhMRvLiqOMlHPhJM4TaJCjHEAJQQJ99BFACi5YpzXJ3w3AAAbACOGt27F"
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "northeurope"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'en',
    'to': ['fr', 'zu', 'ko']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
body = [{
    'text': 'I would really like to drive your car around the block a few times!'
}]

request = requests.post(constructed_url, params=params, headers=headers, json=body)
response = request.json()

# text만 추출
texts = [t['text'] for t in response[0]['translations']]

# 출력
for text in texts:
    print(text)

J’aimerais vraiment conduire votre voiture autour du pâté de maisons plusieurs fois !
Ngingathanda ukushayela imoto yakho ezungeze ibhlokhi izikhathi ezimbalwa!
나는 정말로 당신의 차를 몇 번 블록 주위에 운전하고 싶습니다!


### multi language translator in gradio

In [27]:
import gradio as gr

def translate_with_new_function(input_text):
    # 새 변수 이름 (기존과 충돌 방지)
    subscription_key = "5BOOH90Qyj7T9opgcLQs1dFZWcfHhMRvLiqOMlHPhJM4TaJCjHEAJQQJ99BFACi5YpzXJ3w3AAAbACOGt27F"
    service_endpoint = "https://api.cognitive.microsofttranslator.com"
    service_region = "northeurope"

    api_path = '/translate'
    url = service_endpoint + api_path

    query_params = {
        'api-version': '3.0',
        'from': 'en',
        'to': ['fr', 'zu', 'ko']
    }

    request_headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': service_region,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    request_body = [{'text': input_text}]

    response_obj = requests.post(url, params=query_params, headers=request_headers, json=request_body)
    response_json = response_obj.json()

    translated_texts = [item['text'] for item in response_json[0]['translations']]
    return translated_texts

gr.Interface(
    fn=translate_with_new_function,
    inputs="text",
    outputs="text",
    title="Multi-language Translator"
).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
